# iForest

In [1]:
import warnings
warnings.filterwarnings('ignore')

## General libraries

In [2]:
import os
import sys

import pandas as pd
import numpy as np

from os.path import join
import json
import datetime

import shap
from shap_selection import feature_selection

import time

### Load enviroment variables

In [3]:
from dotenv import load_dotenv
load_dotenv('../.env')

code_root = os.environ['CODE_ROOT']
cfg_path = os.environ['CFG_PATH']
data_root = os.environ['DATA_ROOT']

sys.path.insert(0, code_root)

### Specific libraries

In [4]:
from src.load.functions import get_fs_dataset, fs_datasets_hyperparams
from src.feature_selection.functions import fs_iforest_with_shap, shap_ranks, process_fi

## General parameters

In [5]:
n_iter_fs = 1
n_iter = 1
contamination_percentage = [0.8] 
trees = [25, 50, 100]
group = 0

## Credit card

**Dataset source**: https://www.kaggle.com/mlg-ulb/creditcardfraud

**Additional sources:**

Andrea Dal Pozzolo, Olivier Caelen, Reid A. Johnson and Gianluca Bontempi. Calibrating Probability with Undersampling for Unbalanced Classification. In Symposium on Computational Intelligence and Data Mining (CIDM), IEEE, 2015

Dal Pozzolo, Andrea; Caelen, Olivier; Le Borgne, Yann-Ael; Waterschoot, Serge; Bontempi, Gianluca. Learned lessons in credit card fraud detection from a practitioner perspective, Expert systems with applications,41,10,4915-4928,2014, Pergamon

Dal Pozzolo, Andrea; Boracchi, Giacomo; Caelen, Olivier; Alippi, Cesare; Bontempi, Gianluca. Credit card fraud detection: a realistic modeling and a novel learning strategy, IEEE transactions on neural networks and learning systems,29,8,3784-3797,2018,IEEE

Dal Pozzolo, Andrea Adaptive Machine learning for credit card fraud detection ULB MLG PhD thesis (supervised by G. Bontempi)

Carcillo, Fabrizio; Dal Pozzolo, Andrea; Le Borgne, Yann-Aël; Caelen, Olivier; Mazzer, Yannis; Bontempi, Gianluca. Scarff: a scalable framework for streaming credit card fraud detection with Spark, Information fusion,41, 182-194,2018,Elsevier

Carcillo, Fabrizio; Le Borgne, Yann-Aël; Caelen, Olivier; Bontempi, Gianluca. Streaming active learning strategies for real-life credit card fraud detection: assessment and visualization, International Journal of Data Science and Analytics, 5,4,285-300,2018,Springer International Publishing

Bertrand Lebichot, Yann-Aël Le Borgne, Liyun He, Frederic Oblé, Gianluca Bontempi Deep-Learning Domain Adaptation Techniques for Credit Cards Fraud Detection, INNSBDDL 2019: Recent Advances in Big Data and Deep Learning, pp 78-88, 2019

Fabrizio Carcillo, Yann-Aël Le Borgne, Olivier Caelen, Frederic Oblé, Gianluca Bontempi Combining Unsupervised and Supervised Learning in Credit Card Fraud Detection Information Sciences, 2019

Yann-Aël Le Borgne, Gianluca Bontempi Machine Learning for Credit Card Fraud Detection - Practical Handbook

In [6]:
dataset_id = 'bank'
data = get_fs_dataset(dataset_id, data_root)
hyper = fs_datasets_hyperparams(dataset_id)
data.shape

(41188, 11)

In [7]:
pd.pivot_table(data,
             values = 'job',
               index = 'y', 
              aggfunc = 'count')

,job
y,
0.0,36548
1.0,4640


In [8]:
excluded_cols = []

In [9]:
data

,job,marital,education,default,housing,loan,contact,month,day_of_week,poutcome,y
0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,6.0,1.0,1.0,0.0
1,7.0,1.0,3.0,1.0,0.0,0.0,1.0,6.0,1.0,1.0,0.0
2,7.0,1.0,3.0,0.0,2.0,0.0,1.0,6.0,1.0,1.0,0.0
3,0.0,1.0,1.0,0.0,0.0,0.0,1.0,6.0,1.0,1.0,0.0
4,7.0,1.0,3.0,0.0,0.0,2.0,1.0,6.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
41183,5.0,1.0,5.0,0.0,2.0,0.0,0.0,7.0,0.0,1.0,1.0
41184,1.0,1.0,5.0,0.0,0.0,0.0,0.0,7.0,0.0,1.0,0.0
41185,5.0,1.0,6.0,0.0,2.0,0.0,0.0,7.0,0.0,1.0,0.0
41186,9.0,1.0,5.0,0.0,0.0,0.0,0.0,7.0,0.0,1.0,1.0


### iForest

## Parameters

In [10]:
path_fi_shap = os.path.join(data_root, "outputs", f"{dataset_id}_fi_shap")

## Feature selection by SHAP

In [11]:
fs_shap, fi_shap, _ = fs_iforest_with_shap(data, contamination_percentage=contamination_percentage, excluded_cols=excluded_cols, n_iter_fs=n_iter_fs)

100%|===================| 8230/8238 [00:53<00:00]        

In [12]:
fi_shap_all = process_fi(fi_shap, 10)

In [13]:
fi_shap_all

,n_feats,cum_value,cum_value_percentage,n_feats_percentage,feat_selected
0,1,0.535348,17.549251,10.0,[default]
1,2,1.001129,32.818013,20.0,"[default, loan]"
2,3,1.425444,46.727479,30.0,"[default, loan, marital]"
3,4,1.739739,57.030375,40.0,"[default, loan, marital, poutcome]"
4,5,2.027879,66.475911,50.0,"[default, loan, marital, poutcome, contact]"
5,7,2.541615,83.316670,70.0,"[default, loan, marital, poutcome, contact, mo..."
6,8,2.779434,91.112621,80.0,"[default, loan, marital, poutcome, contact, mo..."
7,10,3.050547,100.000000,100.0,"[default, loan, marital, poutcome, contact, mo..."
8,10,3.050547,100.000000,100.0,"[default, loan, marital, poutcome, contact, mo..."


In [14]:
fi_shap_all.to_parquet(path_fi_shap)